# Implementación del método de Naive para identificar objetos en movimiento

**Índice**   
1. [Registro de frame](#id1)
2. [Funciones para implementación](#id2)
3. [Pruebas por frame](#id3)
4. [Segmentación de objetos en movimiento](#id4)
5. [Comparativa con métodos de mezclas gaussianas](#id5)
   


In [1]:
# Librerías principales
import numpy as np
import cv2 as cv
import matplotlib
from matplotlib import pyplot as plt

# Tipo de visualización
%matplotlib inline

# Versiones de librerías
print("".join(f"{x[0]}: {x[1]}\n" for x in [
    ("Numpy",np.__version__),
    ("openCV",cv.__version__),
    ("Matplotlib",matplotlib.__version__),
]))

Numpy: 1.22.3
openCV: 4.5.5
Matplotlib: 3.5.1



In [2]:
# Definición de la ruta para levantar los videos
VD_DIR = r'.\videos'
VD_NAME = 'vtest.avi'


## Registro de frames<a name="id1"></a>
Se levanta el video que se empleará como test.


In [3]:
# Creación del objeto para luego manipular los frames
#-------------------
capture = cv.VideoCapture(os.path.join(VD_DIR, VD_NAME))

if not capture.isOpened:
    print('Falla al abrir el archivo: ' + VD_NAME)
    exit(0)

Cada frame se guarda en un array. Se tratará como un tensor. La primera dimensión da cuenta de la cantidad de frame que posee el video y fueron guardados de esta forma. Luego se extraen valores a emplear luego.|

In [4]:
# Guardad de cada frame en array
frames = []
while True:
    read, frame= capture.read()
    if not read:
        break
    frames.append(frame)
frames = np.array(frames)

# Frames por segundo
fps = capture.get(cv.CAP_PROP_FPS)
# Cantidad de frames guardados como array
cant_frames = len(frames)

# Verificación de lectura correcta
if int(capture.get(cv.CAP_PROP_FRAME_COUNT)) == len(frames):
    print("La cantidad de frames guardados es correcta")
    print("Cantidad de frames registrados: ", cant_frames)
    print("Verificación de dimensiones: ", frames.shape)
else:
    print("No se ha guardado correctamente la cantidad de frames")



La cantidad de frames guardados es correcta
Cantidad de frames registrados:  795
Verificación de dimensiones:  (795, 576, 768, 3)


## Funciones para implementación<a name="id2"></a>
La implementación se separó en dos partes. La primera es la construcción del background. Se calcula como la mediana de una muestra aleatoria de frames. El cálculo es por canal y por pixel. Luego se arma la máscara o _foreground_. Para ello se toma el frame que se está leyendo en el momento de la reproducción del video. Se la mejora aplicando en pasos:
- Binarización tipo Otsu.
- Filtro morfológico de cierre, ya que conviene cerrar la figura para que no haya agujeros (ceros) dentro de los objetos que afecten la transformada.


In [5]:
def background_naive(input_frames, batch_size, seed, m_type='float32'):
    '''
    Función que devuelve la mediana, por canal, de un batch random formado por los frames de un video
    Los valores son de tipo flotante por defecto.
    - input_frames: array que contiene los frames de video a procesar.
    - batch_size: tamaño del batch que contiene las muestras seleccionadas aleatoriamente sin reemplazo.
    - seed: semilla para garantizar la repetitividad del proceso.
    '''
    rng = np.random.default_rng(seed)
    # Cantidad de frames a procesar
    cant_frames = len(input_frames)
    # Generación de enteros random
    idx = rng.choice(cant_frames, size=batch_size, replace=False, shuffle=False)
    # Armado del batch. Se convierte a tipo flotante
    batch = input_frames[idx,...].astype(m_type)

    # Mediana del batch
    return np.median(batch, axis=0)

In [6]:
def foreground_naive(background, input_frame, thresh_value=0, max_val=255):
    '''
    Función que devuelve la máscara del objeto que se encuentra en movimiento. Incluye la binarizacion, empleando el método Otsu,
    y la aplicación del filtro morfológico tipo cierre.
    - background: tensor de 3 canales con la mediana del batch aleatorio de frames
    - frame: frames de 3 canales con objetos a detectar.
    - tresh_value: valor umbral del método OTSHU.
    - max_val: valor máximo del método OTSHU.
    '''
    # Conversión del frame a float32
    input_frame.astype('float32')
    # resta
    diff = background - input_frame
    # normalizado
    diff = cv.normalize(diff,None,0,255,cv.NORM_MINMAX).astype('uint8')
    # pasaje a escala de grises
    diff_gray = cv.cvtColor(diff,cv.COLOR_BGR2GRAY)
    # binarización otsu
    ret, thresh = cv.threshold(diff_gray,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
    # Creamos un elemento estructurante y aplicamos operaciones morfologicas
    kernel = np.ones((3,3),np.uint8)
    # Filtro morfológico tipo cierre.
    closing = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel, iterations = 3)

    return closing
     

## Pruebas por frame<a name="id3"></a>

In [7]:
# # Selección de un frame del video: cambiar el número del primer índice
# f_p = frames[100,...]
# f_p.shape

In [8]:
# # background
# background_p = background_naive(frames, 50, 10)
# # foreground
# mask = foreground_naive(background_p, f_p)

# plt.imshow(mask, cmap='gray')
# plt.show()

In [9]:
# # Implementación de máscara
# indices = np.where(mask==255)
# f_p[indices[0], indices[1], :] = [200, 0, 255]
# plt.imshow(f_p)
# plt.show()

## Segmentación de objetos en movimiento<a name="id4"></a>
A partir de las funciones presentadas, se segmentó el viceo de los elementos en movimiento a partir de la sustracción de fondo. En este caso particular se trató de personas. Los parámetros a definir previamente son:
- El tamaño del batch de frames.
- El tiempo de actualización del background.
- El valor de la semilla para garantizar la repetitibilidad de la prueba. 
  
Con los valores predefinidos, la implementación muestra buenos resultados de segmentación. Solo si se debe mencionar que no logra incluir la mayoría de los rostros en la segmentación. Ocurre también que, dependiendo el color de ropa, tampoco segmenta a la vestimenta en particular. Sin embargo son casos marginales y la figura principal en movimiento es detectada por completo. 

En cuanto a performance al correr el algoritmo, presenta el inconveniente de la actualización del background. Su ejecución presenta un tiempo muerto, igual al necesario para llevar a cabo la actualización del fondo del background.

En la carpeta _result_ se encuentra grabado el video con el resultado final de la segmentación por substracción de fondo.


In [10]:
# Parámetros a definir para la segmentación
# -----------------------------------------
batch_size = 50 # tamaño del batch de frames para armar el background
t = 10.0 # tiempo de muestreo
seed = 122 # semilla para garantizar la repetitibilidad de la prueba


In [11]:
# Implementación de la segmentación de objetos en movimiento por Naive
# --------------------------------------------------------------------

# Definición de las posiciones de frame donde se realiza la actualización del background
update = np.arange(0, cant_frames, int(t*fps))
update = np.append(update,1) # se agrega un 1 al final para que sean coherentes las dimensiones, no afecta a la implementación

# Generación de semillas para garantizar la repetitibilidad de background generados aleatoriamente
rng = np.random.default_rng(seed)
seed_bg = rng.choice(4*len(update), size=len(update), replace=False, shuffle=False) 

# Contadores
count=0
s=0

# Creación del objeto para aplicar Naive a los frames
cap_naive = cv.VideoCapture(os.path.join(VD_DIR, VD_NAME))
if not cap_naive.isOpened:
    print('Falla al abrir el archivo: ' + VD_NAME)
    exit(0)

# Parámetros y objetos necesarios para grabar la ejecución (descomentar para grabación)
# -------------------------------------------------------------------------------------
# width = int(cap_naive.get(cv.CAP_PROP_FRAME_WIDTH))
# height = int(cap_naive.get(cv.CAP_PROP_FRAME_HEIGHT))
# size = (width, height)
# fourcc = cv.VideoWriter_fourcc(*'XVID')
# out_seg_naive = cv.VideoWriter(r'.\result\seg_naive.avi', fourcc, fps, size)


# Ciclo de segmentación por substracción Naive
# --------------------------------------------
while True:
    # Lectura de un frame
    r_n, f_n = cap_naive.read()
    if not r_n:
        break
    
    # Actualización del background
    if count == update[s]:
        background = background_naive(frames, batch_size, seed_bg[s])
        s+=1
    # Contrucción del foreground 
    fg_n = foreground_naive(background, f_n)
    # Aplicación de la máscara al frame que se está leyendo
    idx_fg_n = np.where(fg_n==255)
    f_n[idx_fg_n[0], idx_fg_n[1], :] = [200, 0, 255]
    # Registro sobre la imagen del número de frame procesado
    cv.rectangle(f_n, (10, 2), (100,20), (255,255,255), -1)
    cv.putText(f_n, str(cap_naive.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))

    # Salida foreground y frame segmentado
    cv.imshow('Foreground Naive', fg_n)
    cv.imshow('Segmentado Naive', f_n)
    # Grabación (descomentar para grabar)
    # out_seg_naive.write(f_n)
    
    count+=1

    # Se corre el video hasta que termine o se apriete escape. Aprentando 's' se guarda el frame actual
    keyboard = cv.waitKey(30)
    if keyboard == 'q' or keyboard == 27:
        break
    elif keyboard == ord('s'):
        cv.imwrite(r'\result\frame_seg_naive.png',f)

cv.destroyAllWindows()
# Descomentar si se realiza grabación
# out_seg_naive.release()
cap_naive.release()

## Comparativa con el método de mezclas gaussianas<a name="id5"></a>
Se aplica el algoritmo **MOG2** al video. Se basa en la segmentación de fondo/primer plano basado en una mezcla de gaussianas. Este se encarga de seleccionar el número apropiado de distribución gaussiana para cada píxel. También MOG utiliza un método para modelar cada píxel de fondo mediante una mezcla de distribuciones K gaussianas (K es de 3 a 5). MOG2 selecciona el número apropiado de gaussianas para cada pixel automáticamente. 

En cuanto a resultados, se tiene que:
- La calidad de segmentación no fue tan buena como con la implementación de Naive.
- La performance de la ejecución del algoritmo si resultó mucho mejor. No se presenta el inconveniente del tiempo muerto por actualización del background.

En la carpeta _result_ se encuentra grabado el video con el resultado final de la segmentación aplicando MOG2. 

In [12]:
# Creación del objeto para aplicar Naive a los frames
cap_MOG2 = cv.VideoCapture(os.path.join(VD_DIR, VD_NAME))
if not cap_MOG2.isOpened:
    print('Falla al abrir el archivo: ' + VD_NAME)
    exit(0)

# Parámetros y objetos necesarios para grabar la ejecución (descomentar para grabación)
# Se debe hacer una grabación de video por vez
# -------------------------------------------------------------------------------------
# width = int(cap_MOG2.get(cv.CAP_PROP_FRAME_WIDTH))
# height = int(cap_MOG2.get(cv.CAP_PROP_FRAME_HEIGHT))
# size = (width, height)
# fourcc = cv.VideoWriter_fourcc(*'XVID')
# out_seg_MOG2 = cv.VideoWriter(r'.\result\seg_MOG2.avi', fourcc, fps, size)

# Creación del objeto openCV para substracción de fondo por MOG2
backSub = cv.createBackgroundSubtractorMOG2()

# Ciclo de segmentación por susbtracción MOG2
#--------------------------------------------
while True:
    # Lectura de un frame
    r_MOG2, f_MOG2 = cap_MOG2.read()
    if not r_MOG2:
        break
    
    # Construcción del foreground
    fg_MOG2 = backSub.apply(f_MOG2)
    # Aplicación de la máscara al frame que se está leyendo
    idx_fg_MOG2 = np.where(fg_MOG2==255)
    f_MOG2[idx_fg_MOG2[0], idx_fg_MOG2[1], :] = [200, 0, 255]
    # Registro sobre la imagen del número de frame procesado
    cv.rectangle(f_MOG2, (10, 2), (100,20), (255,255,255), -1)
    cv.putText(f_MOG2, str(cap_MOG2.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))

    # Salida foreground y frame segmentado
    cv.imshow('Foreground MOG2', fg_MOG2)
    cv.imshow('Segmentado MOG2', f_MOG2)
    # Grabación (descomentar para grabar)
    # out_seg_MOG2.write(f_MOG2)
    
    # Se corre el video hasta que termine o se apriete escape. Aprentando 's' se guarda el frame actual
    keyboard = cv.waitKey(30)
    if keyboard == 'q' or keyboard == 27:
        break
    elif keyboard == ord('s'):
        cv.imwrite(r'\result\frame_seg_MOG2.png',f)

cv.destroyAllWindows()
# Descomentar si se realiza grabación
# out_seg_MOG2.release()
cap_MOG2.release()